In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [13]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [1]:
!pip install snowflake-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 kB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 14.8 MB/s eta 0:00:0000:0100:01


In [14]:
import pandas as pd
import snowflake.connector

conn = snowflake.connector.connect(
    user='XXXXXXXXXX',
    password='XXXXXXXXXXXXXX',
    account='XXXXXXXXXXXXXXXX',
    database = 'SNOWFLAKE_LEARNING_DB',
    schema = 'PUBLIC',
    warehouse='COMPUTE_WH',
)

In [15]:
query = 'select * from "HOUSE_DATA"'
df = pd.read_sql(query, conn)
conn.close()


/tmp/ipykernel_55/538938221.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [16]:
df.head()

,ID,DATE,PRICE,BEDROOMS,BATHROOMS,SQFT_LIVING,SQFT_LOT,FLOORS,WATERFRONT,VIEW,...,GRADE,SQFT_ABOVE,SQFT_BASEMENT,YR_BUILT,YR_RENOVATED,ZIPCODE,LAT,LONG,SQFT_LIVING15,SQFT_LOT15
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [17]:
df.columns

Index(['ID', 'DATE', 'PRICE', 'BEDROOMS', 'BATHROOMS', 'SQFT_LIVING',
       'SQFT_LOT', 'FLOORS', 'WATERFRONT', 'VIEW', 'CONDITION', 'GRADE',
       'SQFT_ABOVE', 'SQFT_BASEMENT', 'YR_BUILT', 'YR_RENOVATED', 'ZIPCODE',
       'LAT', 'LONG', 'SQFT_LIVING15', 'SQFT_LOT15'],
      dtype='object')

In [18]:
features = ['BEDROOMS', 'BATHROOMS', 'SQFT_LIVING',
       'SQFT_LOT', 'FLOORS', 'WATERFRONT', 'VIEW', 'CONDITION', 'GRADE',
       'SQFT_ABOVE', 'SQFT_BASEMENT', 'YR_BUILT', 'YR_RENOVATED', 'ZIPCODE',
       'LAT', 'LONG', 'SQFT_LIVING15', 'SQFT_LOT15']

In [19]:
x = df[features]

In [20]:
y = df['PRICE']

In [21]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

In [22]:
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", Ridge())
])


In [23]:
param_grid = [
    # Ridge
    {
        "model": [Ridge()],
        "model__alpha": [0.01, 0.1, 1, 10, 100]
    },

    # Lasso
    {
        "model": [Lasso(max_iter=5000)],
        "model__alpha": [0.01, 0.1, 1, 10, 100]
    },

    # ElasticNet
    {
        "model": [ElasticNet(max_iter=5000)],
        "model__alpha": [0.01, 0.1, 1, 10],
        "model__l1_ratio": [0.2, 0.5, 0.8]
    }
]


In [24]:
grid = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)


In [25]:
grid.fit(x_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.690e+13, tolerance: 1.807e+11
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.530e+13, tolerance: 1.728e+11
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.702e+13, tolerance: 1.834e

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', Ridge())]),
             n_jobs=-1,
             param_grid=[{'model': [Ridge()],
                          'model__alpha': [0.01, 0.1, 1, 10, 100]},
                         {'model': [Lasso(max_iter=5000)],
                          'model__alpha': [0.01, 0.1, 1, 10, 100]},
                         {'model': [ElasticNet(max_iter=5000)],
                          'model__alpha': [0.01, 0.1, 1, 10],
                          'model__l1_ratio': [0.2, 0.5, 0.8]}],
             scoring='neg_root_mean_squared_error')

In [26]:
print("Best Model:", grid.best_estimator_)
print("Best Params:", grid.best_params_)


Best Model: Pipeline(steps=[('scaler', StandardScaler()),
                ('model', ElasticNet(alpha=0.01, l1_ratio=0.2, max_iter=5000))])
Best Params: {'model': ElasticNet(max_iter=5000), 'model__alpha': 0.01, 'model__l1_ratio': 0.2}
